# Chapter 5: Brute-Force Algorithm Validation

## Introduction
In this chapter, we will subject our signature algorithm to a rigorous validation test. The objective is to verify if it can correctly discriminate between all graph structures for a given order `n` by checking it against established mathematical results.

### The Fundamental Hypothesis

A canonical signature algorithm is correct if and only if it meets two fundamental conditions:
1.  **Consistency**: It must assign the same signature to two graphs that are structurally identical (*isomorphic*).
2.  **Uniqueness**: It must assign different signatures to two graphs that are structurally different (*non-isomorphic*).

### Testing Methodology

Rather than starting with a pre-filtered list of unique graphs, we will adopt a "brute-force" approach that tests both conditions simultaneously. This method is more comprehensive as it makes no initial assumptions about the graphs.

The process is as follows:
1.  For a given order `n`, we determine the number of possible edges, `k = n * (n - 1) / 2`.
2.  We generate all **`2^k`** possible labeled graphs. This set contains every possible combination of edges between the `n` vertices.
3.  For each of these graphs, we calculate its canonical signature using our algorithm.
4.  We use a `set` to count the number of unique signatures obtained at the end of the process.

### Validation Criterion

The test is considered a **success** if the number of unique signatures we discover is exactly equal to the known number of non-isomorphic graphs for order `n`. This reference value is provided by the **OEIS A000088** sequence (On-Line Encyclopedia of Integer Sequences).

### Scope of the Experiment and Computational Cost

This method is exhaustive, but its computational cost increases exponentially. We will therefore begin by validating our algorithm for orders `n` from 3 to 6. The table below illustrates the explosion in the number of cases to be processed.

| Order (n) | Possible Edges (k) | Graphs to Test (2^k)  | Expected Unique Graphs (OEIS A000088) |
| :-------- | :------------------- | :---------------------- | :-------------------------------------- |
| 3         | 3                    | 8                       | 4                                       |
| 4         | 6                    | 64                      | 11                                      |
| 5         | 10                   | 1,024                   | 34                                      |
| 6         | 15                   | 32,768                  | 156                                     |
| 7* | 21                   | 2,097,152               | 1,044                                   |
| 8* | 28                   | 268,435,456             | 12,005                                  |
| 9* | 36                   | 68,719,476,736          | 274,668                                 |
| 10*| 45                   | ~3.5 x 10^13            | 12,005,168                              |

*Orders marked with an asterisk require considerable processing time.*



## 5.1 Order 3

In [21]:
from collections import defaultdict
from utils import get_signature, check

# The 8 possible labeled graphs of order 3
graphs_order_3_g6 = [
    'B?',  # 0 edges (1 graph)
    'BQ', 'BS', 'Bw',  # 1 edge (3 graphs)
    'BR', 'Bi', 'Bq',  # 2 edges (3 graphs)
    'B~'   # 3 edges (1 graph)
]

# Dictionary to store the results: {signature: [list_of_g6_strings]}
signature_groups = defaultdict(list)

print(f"Processing {len(graphs_order_3_g6)} graphs of order 3...")

# Loop through all 8 graphs
for g6 in graphs_order_3_g6:
    gs = get_signature(g6)
    compact_signature = gs.sig()
    signature_groups[compact_signature].append(g6)
    print(f"g6:{g6} sig:{compact_signature}")

# --- Analysis of Results ---

num_unique_found = len(signature_groups)
num_expected = 4  # From OEIS A000088 for n=3

print("--- Results for n=3 ---")

check(
    f"We expect {num_expected} unique signatures and got {num_unique_found}", num_unique_found == num_expected)

Processing 8 graphs of order 3...
g6:B? sig:[{nc:0},{nc:0},{nc:0}]
g6:BQ sig:[{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:BS sig:[{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:Bw sig:[{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]}]},{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]}]},{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]}]}]
g6:BR sig:[{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:Bi sig:[{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]}]
g6:Bq sig:[{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]}]
g6:B~ sig:[{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{

## 5.2 Order 4

In [22]:
import itertools
import networkx as nx


def test_order(order_to_test, num_expected, print_groups):
    nodes_n = list(range(order_to_test))

    possible_edges = list(itertools.combinations(nodes_n, 2))
    signature_groups = defaultdict(int)
    graphs_generated_count = 0
    total_graphs_to_generate = 2**len(possible_edges)

    modulo = 2500
    while total_graphs_to_generate / modulo > 100:
        modulo *= 10

    for num_edges in range(len(possible_edges) + 1):
        for edge_combo in itertools.combinations(possible_edges, num_edges):
            current_nx_graph = nx.Graph()
            current_nx_graph.add_nodes_from(nodes_n)
            current_nx_graph.add_edges_from(list(edge_combo))
            graphs_generated_count += 1
            gs = get_signature(current_nx_graph)
            compact_signature_for_graph = gs.sig()
            signature_groups[compact_signature_for_graph] += 1
            if (graphs_generated_count % modulo == 0):
                print(f"[{(100 * graphs_generated_count) // total_graphs_to_generate}%] Found {len(signature_groups)} unique signatures out of {num_expected}")

    num_unique_found_n4 = len(signature_groups)
    check(
        f"We expect {num_expected} unique signatures and got {num_unique_found_n4}", num_unique_found_n4 == num_expected
    )
    if print_groups:
        for i, (sig_str, count_value) in enumerate(sorted(signature_groups.items())): 
            print(
                f"Signature Group {i+1}: {sig_str} -> Count: {count_value}"
            )

test_order(4, num_expected=11, print_groups=True)

✅ We expect 11 unique signatures and got 11
Signature Group 1: [{nc:0},{nc:0},{nc:0},{nc:0}] -> Count: 1
Signature Group 2: [{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0},{nc:0}] -> Count: 6
Signature Group 3: [{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]}] -> Count: 3
Signature Group 4: [{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:0,fi:3,rs:1}] -> Count: 12
Signature Group 5: [{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,n:[{nc:2,ll:2}]}]},{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,n:[{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,ll:2}]}]},{nc:1,n:[{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,ll:2}]}]}] -> Count: 12
Signature Group 6: [{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:

## 5.3 Order 5

In [23]:
test_order(5, num_expected=34, print_groups=True)

✅ We expect 34 unique signatures and got 34
Signature Group 1: [{nc:0},{nc:0},{nc:0},{nc:0},{nc:0}] -> Count: 1
Signature Group 2: [{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0},{nc:0},{nc:0}] -> Count: 10
Signature Group 3: [{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0,fi:4,rs:1}] -> Count: 15
Signature Group 4: [{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:0},{nc:0}] -> Count: 30
Signature Group 5: [{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]}] -> Count: 30
Signature Group 6: [{nc:2,fi:0,rs:3,n:[{nc:2},{nc:2}]},{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:1,n:[{nc:2,ll:2}]}]}]},{nc:1,n:[{nc:2,ll:2}]}]},{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:1,n:[{nc:2,ll:2}]}]}]},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,n:[{nc:2,n:[{nc

## 5.4 Order 6

In [20]:
test_order(6, num_expected=156, print_groups=False)

[7%] Found 33 unique signatures out of 156
[15%] Found 39 unique signatures out of 156
[22%] Found 54 unique signatures out of 156
[30%] Found 61 unique signatures out of 156
[38%] Found 78 unique signatures out of 156
[45%] Found 78 unique signatures out of 156
[53%] Found 102 unique signatures out of 156
[61%] Found 102 unique signatures out of 156
[68%] Found 102 unique signatures out of 156
[76%] Found 123 unique signatures out of 156
[83%] Found 123 unique signatures out of 156
[91%] Found 138 unique signatures out of 156
[99%] Found 152 unique signatures out of 156
✅ We expect 156 unique signatures and got 156


## 5.5 Order 7

In [24]:
test_order(7, num_expected=1044, print_groups=False)

[1%] Found 40 unique signatures out of 1044
[2%] Found 81 unique signatures out of 1044
[3%] Found 81 unique signatures out of 1044
[4%] Found 146 unique signatures out of 1044
[5%] Found 146 unique signatures out of 1044
[7%] Found 146 unique signatures out of 1044
[8%] Found 146 unique signatures out of 1044
[9%] Found 204 unique signatures out of 1044
[10%] Found 243 unique signatures out of 1044
[11%] Found 243 unique signatures out of 1044
[13%] Found 243 unique signatures out of 1044
[14%] Found 243 unique signatures out of 1044
[15%] Found 243 unique signatures out of 1044
[16%] Found 243 unique signatures out of 1044
[17%] Found 243 unique signatures out of 1044
[19%] Found 243 unique signatures out of 1044
[20%] Found 374 unique signatures out of 1044
[21%] Found 374 unique signatures out of 1044
[22%] Found 374 unique signatures out of 1044
[23%] Found 374 unique signatures out of 1044
[25%] Found 374 unique signatures out of 1044
[26%] Found 374 unique signatures out of 1044